In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as stats
import statsmodels.tsa.stattools as sts
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats.distributions import chi2
import os

from pandas.tseries.offsets import CustomBusinessDay

pd.set_option('display.max_columns', None)

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_excel("./lecturas.xlsx")

In [ ]:
df.head()

In [ ]:
df.shape

# Tratamiento de nulos

In [ ]:
df.isnull().sum()

In [ ]:
print('valores nulos: ', df.isnull().sum().sum())

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['SOCIO', 'N_SOCIO', 'APELLIDOS', 'NOMBRES', 'DIRECCION', 'NRO', 'FONO',
       'RUC_CLI', 'COD', 'CREDITO', 'LITE', 'EMISION', 'MEDIDOR', 'ALUMBRADO', 'BASURA', 'CERTIF', 'TITULO', 'BANDERA', 'CARNET', 'MINIMO', 'MULTAS', 
        'RECONEX', 'BANDI', 'OTROS', 'ALCANTARI', 'ORDENES', 'HEXA',
       'NUMFAC', 'FACTURA', 'COPIA', 'REUNION', 'TRABAJOS', 'CPU'])

In [ ]:
df.tail(30)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df = df.dropna(subset=['IMPORTE', 'CATEGORIA', 'MES_COBRO', 'LEC_ANTES',
       'LECT_HOY', 'CONSUMO', 'ATRASO', 'TOTAL', 'VENCE', 'INGRESO', 'VALOR',
       'FLEC', 'FALTAS', 'BENEMERITO', 'ZONA', 'DESCBENEME', 'PASIBLE',
       'LEIANTES', 'GESTION'])
df.isnull().sum()

In [ ]:
df.shape

# Tratamiento de fechas

In [ ]:
df.dtypes

In [ ]:
df[['VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']] = df[['VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']].astype(int) #para quiar el .0
df[['VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']] = df[['VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']].astype(str)
df.dtypes

In [ ]:
fechas = df['FECHA'].value_counts().to_frame()
fechas.iloc[1020:1080]

In [ ]:
filtered_df = df[(df['FECHA'].str.len() == 8) | (df['FECHA'].isna())]
filtered_df.shape

In [ ]:
vence = filtered_df['VENCE'].value_counts().to_frame()
vence.iloc[180:240]

In [ ]:
filtered_df = filtered_df[(filtered_df['VENCE'].str.len() == 8) | (filtered_df['VENCE'].isna())]
filtered_df.shape

In [ ]:
ingreso = filtered_df['INGRESO'].value_counts().to_frame()
ingreso.iloc[180:240]

In [ ]:
flec = filtered_df['FLEC'].value_counts().to_frame()
#flec.iloc[60:120]
#flec.iloc[120:180]
flec.iloc[180:240]

In [ ]:
filtered_df = filtered_df[(filtered_df['FLEC'].str.len() == 8) | (filtered_df['FLEC'].isna())]
filtered_df.shape

In [ ]:
pasible = filtered_df['PASIBLE'].value_counts().to_frame()
pasible.iloc[180:240]

In [ ]:
filtered_df = filtered_df[(filtered_df['PASIBLE'].str.len() == 8) | (filtered_df['PASIBLE'].isna())]
filtered_df.shape

In [ ]:
leiantes = filtered_df['LEIANTES'].value_counts().to_frame()
leiantes.iloc[180:240]

In [ ]:
filtered_df = filtered_df[(filtered_df['LEIANTES'].str.len() == 8) | (filtered_df['LEIANTES'].isna())]
filtered_df.shape

In [ ]:
filtered_df[['FECHA', 'VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']] = filtered_df[['FECHA', 'VENCE', 'INGRESO', 'FLEC', 'PASIBLE', 'LEIANTES']].astype(float)
filtered_df.dtypes

In [ ]:
fechas_erroneas = filtered_df[(filtered_df['VENCE'] > 20260000) | (filtered_df['VENCE'] < 19500000)]
fechas_erroneas

In [ ]:
filtered_df = filtered_df[(filtered_df['VENCE'] < 20260000) & (filtered_df['VENCE'] > 19500000)]
filtered_df.shape

In [ ]:
fechas_erroneas = filtered_df[(filtered_df['FLEC'] > 20260000) | (filtered_df['FLEC'] < 19500000)]
fechas_erroneas

In [ ]:
filtered_df = filtered_df[(filtered_df['FLEC'] < 20260000) & (filtered_df['FLEC'] > 19500000)]
filtered_df.shape

In [ ]:
fechas_erroneas = filtered_df[(filtered_df['PASIBLE'] > 20260000) | (filtered_df['PASIBLE'] < 19500000)]
fechas_erroneas

In [ ]:
filtered_df = filtered_df[(filtered_df['PASIBLE'] < 20260000) & (filtered_df['PASIBLE'] > 19500000)]
filtered_df.shape

In [ ]:
filtered_df["FECHA"] = pd.to_datetime(filtered_df["FECHA"], format='%Y%m%d')
filtered_df["VENCE"] = pd.to_datetime(filtered_df["VENCE"], format='%Y%m%d')
filtered_df["INGRESO"] = pd.to_datetime(filtered_df["INGRESO"], format='%Y%m%d')
filtered_df["FLEC"] = pd.to_datetime(filtered_df["FLEC"], format='%Y%m%d')
filtered_df["PASIBLE"] = pd.to_datetime(filtered_df["PASIBLE"], format='%Y%m%d')
filtered_df["LEIANTES"] = pd.to_datetime(filtered_df["LEIANTES"], format='%Y%m%d')
filtered_df

# Tratamiento de inconsistencias

In [ ]:
concepto = filtered_df['CONCEPTO'].value_counts().to_frame()
concepto

In [ ]:
filtered_df["CONCEPTO"] = filtered_df["CONCEPTO"].notna().astype(int)
concepto = filtered_df['CONCEPTO'].value_counts().to_frame()
concepto

In [ ]:
filtered_df = filtered_df.rename(columns={'CONCEPTO':'CANCELADO'})
filtered_df.columns

no se elimino ninguna fila, solo se transformo

In [ ]:
importe = filtered_df['CATEGORIA'].value_counts().to_frame()
importe

In [ ]:
correcciones = {
    'DOMESTICA': 'Domestica',
    'COMERCIAL': 'Comercial',
    'especial': 'Especial'
}
filtered_df['CATEGORIA'] = filtered_df['CATEGORIA'].replace(correcciones)
importe = filtered_df['CATEGORIA'].value_counts().to_frame()
importe

no se elimino ninguna fila, solo se corrigio

In [ ]:
zona = filtered_df['ZONA'].value_counts().to_frame()
zona

In [ ]:
filtered_df['ZONA'] = filtered_df['ZONA'].replace('R5', 'R')
zona = filtered_df['ZONA'].value_counts().to_frame()
zona

In [ ]:
filtered_df

# Tratamiento de valores atipicos

In [ ]:
filtered_df[['IMPORTE', 'CONSUMO', 'TOTAL', 'CORTE', 'FALTAS']].describe()

In [ ]:
q1 = filtered_df['TOTAL'].quantile(q=0.25)
q3 = filtered_df['TOTAL'].quantile(q=0.75)
iqr = q3 - q1
print('cuartil 1: ', q1)
print('cuartil 3: ', q3)
print('rango intercuartilico: ', iqr)

In [ ]:
print('limite inferior: ', q1 - 1.5 * iqr)
print('limite superior: ', q3 + 1.5 * iqr)

In [ ]:
fuera = filtered_df[((filtered_df['TOTAL'] < (q1 - 1.5 * iqr)) | (filtered_df['TOTAL'] > (q3 + 1.5 * iqr))) & filtered_df['CANCELADO'] == 1]
fuera

In [ ]:
negativos = filtered_df[filtered_df['TOTAL']<0]
negativos

In [ ]:
total_df = filtered_df[(((filtered_df['TOTAL'] > 0) & (filtered_df['TOTAL'] < (q3 + 1.5 * iqr)))) & filtered_df['CANCELADO'] == 1]
total_df

# Agrupamiento por fechas

In [ ]:
grouped_df = total_df.groupby(['FECHA']).agg({'TOTAL': np.sum}).sort_values(['FECHA'], ascending=True)
grouped_df

In [ ]:
custom_freq = CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri Sat')
grouped_df = grouped_df.asfreq(custom_freq)

In [ ]:
grouped_df.isna().sum()

In [ ]:
grouped_df = grouped_df.fillna(1)
grouped_df

In [ ]:
grouped_df["TOTAL"].plot(figsize=(20,8), title = "Total pagos diario")
plt.show()

In [ ]:
anio = grouped_df.loc['2023-01-01':'2023-12-31']
anio["TOTAL"].plot(figsize=(20,5), title = "Total pagos diario")
plt.show()

In [ ]:
anio_22 = grouped_df.loc['2022-01-01':'2022-12-31']
anio_22["TOTAL"].plot(figsize=(20,5), title = "Total pagos diario")
plt.show()

In [ ]:
anio_24 = grouped_df.loc['2024-01-01':'2024-12-31']
anio_24["TOTAL"].plot(figsize=(20,5), title = "Total pagos diario")
plt.show()

In [ ]:
mes = grouped_df.loc['2023-12-01':'2023-12-31']
mes["TOTAL"].plot(figsize=(20,5), title = "Total pagos diario")
plt.show()

# Tratamiento de valores atipicos en la serie de tiempo

In [ ]:
q1_serie = grouped_df['TOTAL'].quantile(q=0.25)
q3_serie = grouped_df['TOTAL'].quantile(q=0.75)
iqr_serie = q3_serie - q1_serie
print('q1: ', q1_serie)
print('q3: ', q3_serie)
print('iqr: ', iqr_serie)
plt.ylim(0, 3000)
plt.boxplot(grouped_df['TOTAL'])

In [ ]:
lower_bound = q1_serie - 1.5 * iqr_serie
upper_bound = q3_serie + 1.5 * iqr_serie
print('limite inferior: ', lower_bound)
print('limite superior: ', upper_bound)

In [ ]:
fig = plt.figure(figsize=(14, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[7, 1])
ax1 = plt.subplot(gs[0])
ax1.plot(grouped_df)
ax2 = plt.subplot(gs[1])
ax2.boxplot(grouped_df['TOTAL'])
ax1.axhline(upper_bound, color='red')
ax2.axhline(upper_bound, color='red',  label='outliers')
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
fuera = grouped_df[(grouped_df['TOTAL'] > upper_bound)]
fuera.count()

### Winsorization

In [ ]:
q99 = grouped_df['TOTAL'].quantile(q=0.99)
q99

In [ ]:
fuera = grouped_df[(grouped_df['TOTAL'] > q99)]
fuera.count()

In [ ]:
grouped_df.loc[grouped_df['TOTAL'] > q99, 'TOTAL'] = q99
grouped_df["TOTAL"].plot(figsize=(20,10), title = "Total pagos diario")
plt.show()

# Division de los datos

In [ ]:
size = int(len(grouped_df)*0.8)
train_df = grouped_df.iloc[:size]
test_df = grouped_df.iloc[size:]

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
test_df["TOTAL"].plot(figsize=(20,5), title = "Total pagos diario (datos validacion)")
plt.show()

# Estacionariedad

In [ ]:
sts.adfuller(grouped_df.TOTAL)

# Estacionalidad

In [ ]:
aditivo = seasonal_decompose(grouped_df.TOTAL, model="additive", period=26)
aditivo.plot()
plt.show()

# Autocorrelacion

In [ ]:
sgt.plot_acf(grouped_df.TOTAL, lags = 40, zero = False)
plt.ylim(-0.2, 0.5)
plt.show()

In [ ]:
sgt.plot_pacf(grouped_df.TOTAL, lags = 40, zero = False, method = ('ols')) 
plt.ylim(-0.2, 0.5)
plt.show()

# LSTM

In [ ]:
train_max = train_df.max()
train_min = train_df.min()
train_set_scaled = (train_df - train_min)/(train_max - train_min)
test_set_scaled = (test_df - train_min)/(train_max - train_min)
train_set_scaled

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
time_steps = 1

X_train_1, y_train_1 = create_dataset(train_set_scaled, train_set_scaled.TOTAL , time_steps)
X_test_1,  y_test_1  = create_dataset(test_set_scaled,  test_set_scaled.TOTAL, time_steps)

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense      
from keras.layers import LSTM       
from keras.layers import Dropout    

def lstm_architecture(X_data,rate_dropout):
    model = Sequential()

    model.add(LSTM(units = 250, return_sequences = True, input_shape=(X_data.shape[1], X_data.shape[2])))
    model.add(Dropout(rate=rate_dropout))

    model.add(LSTM(units = 250, return_sequences = True))
    model.add(Dropout(rate=rate_dropout))

    model.add(LSTM(units = 250, return_sequences = True))
    model.add(Dropout(rate=rate_dropout))

    model.add(LSTM(units = 250, return_sequences = False))
    model.add(Dropout(rate=rate_dropout))

    model.add(Dense(units = 1))

    model.summary()

    return model

In [ ]:
import datetime
print('Iniciando a las: ', datetime.datetime.now())
print("...")

model_1 = lstm_architecture(X_data = X_train_1, rate_dropout = 0.2)
model_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
history_1 = model_1.fit(X_train_1,
                    y_train_1,
                    epochs=100,
                    batch_size=32,
                    shuffle=False)

print("...")
print('Terminando a las: ', datetime.datetime.now())

In [ ]:
plt.plot(history_1.history['loss'], label='train')
plt.title('Loss function (time_steps = 1)')
plt.show()

In [ ]:
y_pred_1 = model_1.predict(X_test_1)

In [ ]:
y_test_1 = y_test_1 * (train_max[0] - train_min[0]) + train_min[0]    
y_pred_1 = y_pred_1 * (train_max[0] - train_min[0]) + train_min[0]    
y_train_1 = y_train_1 * (train_max[0] - train_min[0]) + train_min[0] 

In [ ]:
plt.figure(num=None, figsize=(25, 10), dpi=80, facecolor='w', edgecolor='k')
plt.plot(np.arange(len(y_train_1), len(y_train_1) + len(y_test_1)), y_test_1.flatten(), marker='.', label="true")
plt.plot(np.arange(len(y_train_1), len(y_train_1) + len(y_test_1)), y_pred_1.flatten(), 'r', marker='.', label="prediction")
#plt.plot(np.arange(0, len(y_train_1)), y_train_1.flatten(), 'g', marker='.', label="history")
plt.ylabel('Count')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(y_test_1, y_pred_1)
rmse = np.sqrt(mse)
print(f'RMSE: ',rmse)

mape = mean_absolute_percentage_error(y_test_1, y_pred_1)
print(f'MAPE: ', mape)

mae = mean_absolute_error(y_test_1, y_pred_1)
print(f'MAE: ',mae)

In [ ]:
time_steps = 6

X_train_6, y_train_6 = create_dataset(train_set_scaled, train_set_scaled.TOTAL , time_steps)
X_test_6,  y_test_6  = create_dataset(test_set_scaled,  test_set_scaled.TOTAL, time_steps)

In [ ]:
model_2 = lstm_architecture(X_data=X_train_6,rate_dropout=0.2)
model_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
history_2 = model_2.fit(X_train_6,
                    y_train_6,
                    epochs=100,
                    batch_size=32,
                    shuffle=False)

print("...")
print('Terminando a las: ', datetime.datetime.now())

In [ ]:
plt.plot(history_2.history['loss'], label='train')
plt.title('Loss function (time_steps = 6)')
plt.show()

In [ ]:
y_pred_6 = model_2.predict(X_test_6)

In [ ]:
y_test_6 = y_test_6 * (train_max[0] - train_min[0]) + train_min[0]    
y_pred_6 = y_pred_6 * (train_max[0] - train_min[0]) + train_min[0]    
y_train_6 = y_train_6 * (train_max[0] - train_min[0]) + train_min[0]  

In [ ]:
plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(np.arange(len(y_train_6), len(y_train_6) + len(y_test_6)), y_test_6.flatten(), marker='.', label="true")
plt.plot(np.arange(len(y_train_6), len(y_train_6) + len(y_test_6)), y_pred_6.flatten(), 'r', marker='.', label="prediction")
#plt.plot(np.arange(0, len(y_train_6)), y_train_6.flatten(), 'g', marker='.', label="history")
plt.ylabel('Count')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(y_test_6, y_pred_6)
rmse = np.sqrt(mse)
print(f'RMSE: ',rmse)

mape = mean_absolute_percentage_error(y_test_6, y_pred_6)
print(f'MAPE: ', mape)

mae = mean_absolute_error(y_test_6, y_pred_6)
print(f'MAE: ',mae)

In [ ]:
time_steps = 26

X_train_26, y_train_26 = create_dataset(train_set_scaled, train_set_scaled.TOTAL , time_steps)
X_test_26,  y_test_26  = create_dataset(test_set_scaled,  test_set_scaled.TOTAL, time_steps)

In [ ]:
model_3 = lstm_architecture(X_data = X_train_26, rate_dropout=0.2)
model_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
history_3 = model_3.fit(X_train_26,
                    y_train_26,
                    epochs=100,
                    batch_size=32,
                    shuffle=False)

print("...")
print('Terminando a las: ', datetime.datetime.now())

In [ ]:
plt.plot(history_3.history['loss'], label='train')
plt.title('Loss function (time_steps = 26)')
plt.show()

In [ ]:
y_pred_26 = model_3.predict(X_test_26)

In [ ]:
y_test_26 = y_test_26 * (train_max[0] - train_min[0]) + train_min[0]    
y_pred_26 = y_pred_26 * (train_max[0] - train_min[0]) + train_min[0]    
y_train_26 = y_train_26 * (train_max[0] - train_min[0]) + train_min[0]  

In [ ]:
plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(np.arange(len(y_train_26), len(y_train_26) + len(y_test_26)), y_test_26.flatten(), marker='.', label="true")
plt.plot(np.arange(len(y_train_26), len(y_train_26) + len(y_test_26)), y_pred_26.flatten(), 'r', marker='.', label="prediction")
#plt.plot(np.arange(0, len(y_train_26)), y_train_26.flatten(), 'g', marker='.', label="history")
plt.title('predicciones y valores reales (time_steps = 26)', size=20)
plt.ylabel('Count')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(y_test_26, y_pred_26)
rmse = np.sqrt(mse)
print(f'RMSE: ',rmse)

mape = mean_absolute_percentage_error(y_test_26, y_pred_26)
print(f'MAPE: ', mape)

mae = mean_absolute_error(y_test_26, y_pred_26)
print(f'MAE: ',mae)

# Prophet

In [ ]:
from prophet import Prophet

In [ ]:
df_prophet = train_df.copy()

In [ ]:
df_prophet = df_prophet.reset_index()
df_prophet = df_prophet.rename(columns={'FECHA':'ds', 'TOTAL':'y'})
df_prophet.head()

In [ ]:
test_prophet = test_df.copy()

In [ ]:
test_prophet = test_prophet.reset_index()
test_prophet = test_prophet.rename(columns={'FECHA':'ds', 'TOTAL':'y'})
test_prophet.head()

In [ ]:
fin_de_año = pd.DataFrame({
    'holiday': 'fin de año',
    'ds': pd.to_datetime(['2021-12-05', '2022-12-05', '2023-12-05', '2024-12-05']),
    'lower_window': -5,
    'upper_window': 5
})

In [ ]:
prophet = Prophet(holidays=fin_de_año, changepoint_range=0.95, yearly_seasonality=50)
prophet.add_seasonality(name='monthly', period=30.5, fourier_order=15)
prophet.fit(df_prophet)

In [ ]:
prophet_pred = prophet.predict(test_prophet)
prophet_pred

In [ ]:
plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(prophet_pred['ds'], test_prophet['y'], label="Valor Real")
plt.plot(prophet_pred['ds'], prophet_pred['yhat'], 'r', label="Prediction")
plt.xlabel('Fecha')
plt.title('predicciones y valores reales Prophet', size=20)
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(test_prophet['y'], prophet_pred['yhat'])
rmse = np.sqrt(mse)
print(f'RMSE: ',rmse)

mape = mean_absolute_percentage_error(test_prophet['y'], prophet_pred['yhat'])
print(f'MAPE: ', mape)

mae = mean_absolute_error(test_prophet['y'], prophet_pred['yhat'])
print(f'MAE: ',mae)

In [ ]:
prophet.plot_components(prophet_pred)